In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from progressbar import ProgressBar

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#progress bar
pbar = ProgressBar()
pbar2 = ProgressBar()
# read the data
x = pd.read_csv('/kaggle/input/tabular-playground-series-apr-2022/train.csv')
y = pd.read_csv('/kaggle/input/tabular-playground-series-apr-2022/train_labels.csv', index_col = 'sequence')
test_set = pd.read_csv('/kaggle/input/tabular-playground-series-apr-2022/test.csv')

Instead of fft, try to convert each sensor in each step group by the sequence & subject

In [ ]:
# Progress to get rid the step column in training set
concat_column_list = [column for column in x.columns if 'sensor' in column or 'sequence' in column]
grouped_x = x.groupby('step')

# generate a new data frame
featured_df = pd.DataFrame()
# copy the sequence & subject column
featured_df['sequence'] = grouped_x.get_group(0)['sequence']
featured_df['subject'] = grouped_x.get_group(0)['subject']
featured_df = featured_df.reset_index(drop= True)
# Merge the 12 sensor conlumn based on the sequence
for i in pbar(range(60)):
    temp_df = grouped_x.get_group(i)[concat_column_list]
    featured_df = featured_df.merge(temp_df, how = 'inner', on='sequence', suffixes = ('',f'_{i+1}'))

In [ ]:
# Progree to get rid the step column in testing set
grouped_test = test_set.groupby('step')

featured_test = pd.DataFrame()
featured_test['sequence'] = grouped_test.get_group(0)['sequence']
featured_test['subject'] = grouped_test.get_group(0)['subject']
featured_test = featured_test.reset_index(drop = True)
for i in pbar2(range(60)):
    temp_df = grouped_test.get_group(i)[concat_column_list]
    featured_test = featured_test.merge(temp_df, how='inner', on='sequence', suffixes = ('', f'_{i+1}'))

In [ ]:
featured_test.head()

In [ ]:
featured_df.head()

Using logistic regression to train & test:

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter = 10000)
model.fit(featured_df, y)
pred = model.predict(featured_test)

result = pd.DataFrame()
result['sequence'] = featured_test.sequence
result['state'] = pred
result.to_csv('submission_3.csv', index = False)
# Score: 0.519

Performance of logistic regression is not great enough. It's similar to the dataset without converting the sensor into column.

Using lightgbm to train and test:

In [ ]:
import lightgbm as lgb

model = lgb.LGBMClassifier(boosting_type = 'dart', learning_rate = 0.1, num_leaves = 60)
model.fit(featured_df, y)
pred = model.predict(featured_test)

result = pd.DataFrame()
result['sequence'] = featured_test.sequence
result['state'] = pred
result.to_csv('submission_5.csv', index = False)
# Scores:0.687

Lightgbm did a better job compared to logistic regression but the performance is not that good compared to fft. Next step is that we can investigate more by adjusting the data with fft or adding more features to provide more information.